# Simple example of time domain beamforming

Let's imagine that we have one or more idealised point sound sources with spherical spreading impinging upon an n-element linear microphone array.

The situation is depicted in the figure below.

![Linear array](figures/linear_array.png)

Note that there are two different frames of reference here. 
There is the frame of reference for the location and orientation of the array.
For this frame of reference we use the unit circle, with the centre of the array at
the origin, and the perpendicular to the array (at $0^o$ rotation) pointing along
the x-axis. 
We use θ to denote the angle of the perpendicular of the array relative to
the unit circle.
For the bearing of contacts relative to the array we use a frame of reference
where $0^o$ denotes pointing directly up the array (forward end-fire),
$90^o$ is pointing perpendicular to the array (broadside) and
$180^o$ is directly down the array (aft end-fire).
We use φ to denote the angle of a signal relative to the array bearing.

Note that we are using point sources, not the plane-wave approximation.

We have used the following helpful links in developing the code forthis example:

* [Time delay estimation for passive sonar signal processing](https://ieeexplore.ieee.org/document/1163560)
* [Jupyter User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html)
* https://en.wikipedia.org/wiki/Wavelength
* [Equations for Plane Waves, Spherical Waves, and Gaussian Beams](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118939154.app3)
* https://github.com/bokeh/bokeh/blob/1.3.2/examples/howto/layouts/dashboard.py
* [Beamforming in the Time Domain](https://onlinelibrary.wiley.com/doi/10.1002/9781119293132.ch11)


For a linear array, a delay of 0 implies a beam steering direction of broadside (perpendicular
to the array).

A delay equivalent to the spacing between two adjacent elements divided by the velocity of
the soundwave implies a beam steering direction of either endfire (parallel to the array),
where the endfire (front or back) is determined by the sign of the delay

More generally, a delay of $d_t = \cos\theta \frac{d}{c}$ corresponds to a beam-steer at
an angle $\theta$ for $\theta \in [0^o, 180^o]$.

Given that we are dealing with discrete time signals, the delay cannot be chosen arbitrarily.

Instead, for sample rate $S$, we have

$$ \cos\theta = \frac{c}{d} d_t = \frac{c}{d} \frac{n}{S} $$

where

$$ -\frac{Sd}{c} < n < \frac{Sd}{c} $$


## Import modules

In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label, Span, Legend, LegendItem
from bokeh.layouts import grid, column, gridplot
from bokeh.models import CustomJS, Slider, ColumnDataSource

import seaborn as sns
output_notebook()

## Set up parameters

In order to create interactive Bokeh widgets, it is necessary to separate the plotting and the 
calculating code, so that an update function can be written, which simple updates the data in
the plots.

See:
* https://github.com/bokeh/bokeh/blob/2.3.0/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb

In [ ]:
# In the code below we choose as input variables, those variables that
# we would like to alter using widgets


verbose = True

#def calculate_everything(Verbose=False):
if True:
    
    # Array characteristics
    num_elements = 6
    spacing = 0.1 # metres
    x_location = 0 # metres
    array_angle = np.deg2rad(20) # degrees
    Arr = []
    array_length = spacing*(num_elements-1)
    for k in range(num_elements):
        x_pos = -(array_length/2 - k*spacing)*np.sin(array_angle)
        y_pos = (array_length/2 - k*spacing)*np.cos(array_angle)
        Arr.append([x_pos,y_pos])
    Arr = np.array(Arr)
    col = sns.color_palette(None, num_elements).as_hex()


    # Source characteristics
    num_sources = 1
    x_src = 1
    amp_src = 1
    wavlength = 0.15
    Src = {}
    Src['position'] = []
    Src['absolute_direction'] = []
    Src['amplitude'] = []
    Src['wavelength'] = []
    for n in range(num_sources):
        (x,y) = x_src, 0.5*n
        Src['position'].append([x,y]) # Source position
        Src['absolute_direction'].append(np.arctan2(y,x))
        Src['amplitude'].append(amp_src) # Source amplitude
        Src['wavelength'].append(wavlength) # Source wavelength

    π = np.pi
    c = 340 # m/s (Speed of sound)

    sample_rate = 96000

    # The time-domain beamforming does not work well if the sample length
    # is an integer multiple of the period of the source signal.
    sample_length = 5.4*Src['wavelength'][0]/c # seconds

    t_0 = 0

    N = 500
    x_min = -1
    x_max = 1
    x = np.linspace(x_min, x_max, N)
    y = np.linspace(x_min, x_max, N)
    t = np.arange(t_0, t_0+sample_length, 1/sample_rate)
    xx, yy = np.meshgrid(x, y)

    Z = 0*xx
    for n in range(num_sources):
        [s_x, s_y] = Src['position'][n] #location of source
        A = Src['amplitude'][n]
        λ = Src['wavelength'][n] # metres
        f = c/λ # Hz

        print(f'Source {n+1}: A = {A}, ',
              f'f = {f} Hz, λ = {λ} m , T = {1/f:0.2e} s') 

        rr = np.sqrt((xx-s_x)*(xx-s_x) + (yy-s_y)*(yy-s_y))

        Z = Z + A/rr*np.sin(2*π/λ*(rr - c*t_0))

    Y = []
    for k in range(num_elements):
        F = 0*t
        for n in range(num_sources):
            [s_x, s_y] = Src['position'][n] #location of source
            A = Src['amplitude'][n]
            λ = Src['wavelength'][n] # metres
            f = c/λ # Hz

            r = np.sqrt((Arr[k,0]-s_x)*(Arr[k,0]-s_x) + 
                        (Arr[k,1]-s_y)*(Arr[k,1]-s_y))
            F = F + A/r*np.sin(2*π/λ*(r - c*t))
        Y.append(F)
        
    #Beamforming
    m = np.arange(int(-np.floor(spacing*sample_rate/c)),int(np.ceil(spacing*sample_rate/c)),1)
    cos_phi = c/spacing*m/sample_rate
    phi = np.arccos(cos_phi)

    look_direction = 15

    beams = 0*phi
    sum_F = {}
    for j in m:
        sum_F[j] = 0*Y[0]
        for k in range(num_elements):
            sum_F[j] = sum_F [j]+ np.roll(Y[k],j*k)
        sum_F[j] = sum_F[j]/num_elements
        beams[j] = np.sum(np.abs(sum_F[j]))/len(sum_F[j])
        
#calculate_everything(True)

In [ ]:
#def plot_everything():
if True:
    
    # Spacial Window
    p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
              )
    p.xaxis.axis_label = p.yaxis.axis_label = 'Distance (m)'
    p.x_range.range_padding = p.y_range.range_padding = 0

    # must give a vector of image data for image parameter
    p.image(image=[Z],
            x=x_min, y=x_min,
            dw=(x_max-x_min), dh=(x_max-x_min), 
            palette="Spectral11", level="image")

    for k in range(num_elements):
        p.circle(Arr[k,0],Arr[k,1],
                 size=10,
                 fill_color=col[k],
                 legend_label=f'Array Element {k}',
                 )

    p.grid.grid_line_width = 0.5
    p.legend.click_policy="hide"

    # Time Series Window
    q = figure()
    q.xaxis.axis_label = 'Time (s)'
    q.yaxis.axis_label = 'Pressure'

    for k in range(num_elements):
        q.line(t,Y[k],color=col[k],legend_label=f'Array Element {k}')

    q.line(t,sum_F[look_direction],color='black',
           legend_label=f'Delay-sum φ:{np.rad2deg(phi[look_direction]):0.2f}°')

    q.legend.click_policy="hide"

    # Beamforming Window

    b = figure()
    b.xaxis.axis_label = 'Steering Angle (degrees)'
    b.yaxis.axis_label = 'Pressure'
    b.circle(np.rad2deg(phi),beams, legend_label='Beampattern')
    b.line(np.rad2deg(phi),beams, legend_label='Beampattern')
    look_line = Span(location=np.rad2deg(phi[look_direction]),
                     dimension='height',
                     line_color='green', line_width=3,
                    )

    for n in range(num_sources):
        source_line = Span(location=90+np.rad2deg(array_angle-Src['absolute_direction'][n]),
                           dimension='height',
                           line_color='red', line_width=2
                          )
    b.line([[], []], legend_label='Look direction', line_color="green", line_width=2)
    b.line([[], []], legend_label='Source direction', line_color="red", line_width=2)
    b.renderers.extend([look_line, source_line])

    # Widget Window
    phi_slider = Slider(start=m[0], end=m[-1], value=1, step=1, title="Look direction")
    array_slider = Slider(start=0, end=180, value=1, step=.1, title="Array direction")
    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

    widgets = column(phi_slider, array_slider, phase_slider, offset_slider)
    
    #return (p, q, b, widgets)

#(p, q, b, widgets) = plot_everything()

show(gridplot([p, q, b, widgets], ncols=2), notebook_handle=True)

In [ ]:
import numpy as np

from bokeh.layouts import grid, column
from bokeh.models import CustomJS, Slider, ColumnDataSource
#from bokeh.plotting import figure, output_file, show

tools = 'pan'


def bollinger():
    # Define Bollinger Bands.
    upperband = np.random.randint(100, 150+1, size=100)
    lowerband = upperband - 100
    x_data = np.arange(1, 101)

    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = figure(x_axis_type='datetime', tools=tools)
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    p.plot_height = 600
    p.plot_width = 800
    p.grid.grid_line_alpha = 0.4
    return [p]


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

    callback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, offset=offset_slider),
                        code="""
        const data = source.data;
        const A = amp.value;
        const k = freq.value;
        const phi = phase.value;
        const B = offset.value;
        const x = data['x']
        const y = data['y']
        for (var i = 0; i < x.length; i++) {
            y[i] = B + A*Math.sin(k*x[i]+phi);
        }
        source.change.emit();
    """)

    amp_slider.js_on_change('value', callback)
    freq_slider.js_on_change('value', callback)
    phase_slider.js_on_change('value', callback)
    offset_slider.js_on_change('value', callback)

    widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = figure(tools=tools)
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
    s2 = figure(tools=tools, x_range=s1.x_range, y_range=s1.y_range)
    s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = figure(tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    return [s1, s2, s3]

l = grid([
    bollinger(),
    slider(),
    linked_panning(),
],)# sizing_mode='stretch_both')

show(l)